# Ray Tune

In [1]:
!pip install -U "ray[tune]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 MB 39.6 MB/s eta 0:00:00


# Env

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
!unzip -qq "/content/drive/MyDrive/MAGISTERKA/datasets/fog-combined.zip" -d "/content/datasets/"

In [4]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Setup

In [5]:
from pathlib import Path
from time import time

import torch
from torchvision.models import get_weight
from torchvision.transforms import v2

import numpy as np
import pandas as pd
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.base import ClassifierMixin
from typing import Any, TypeVar
from collections import namedtuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

import ray
from ray import tune


In [6]:
base_transform = v2.Compose([
    v2.Resize((256, 256), v2.InterpolationMode.BILINEAR),
    v2.CenterCrop((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])
transforms = {
    "grayscale": v2.Compose([
        base_transform,
        v2.Grayscale(num_output_channels=3)
    ]),
    "color": v2.Compose([
        base_transform,
    ]),
}

In [7]:
BASE_PATH = Path("/content")
DRIVE_PATH = Path("/content/drive/MyDrive/MAGISTERKA")

In [8]:
import sys
sys.path.insert(0, str(DRIVE_PATH / 'src'))
from cnn_model import CNNClassifier, get_dataloader

In [9]:
DATASET_PATHS = {
    'fog-detection': BASE_PATH / 'datasets/fog-detection-dataset-prepared',
    'fog-or-smog': BASE_PATH / 'datasets/fog-or-smog-detection-dataset-prepared',
    'foggy-cityscapes': BASE_PATH / 'datasets/foggy-cityscapes-image-dataset-prepared',
    'combined': BASE_PATH / 'datasets/fog-combined',
}

DATASET_NORMALIZATION = {
    'fog-detection': {'mean': [0.4850, 0.5044, 0.4878], 'std': [0.2631, 0.2524, 0.2793]},
    'fog-or-smog': {'mean': [0.5411, 0.5339, 0.5088], 'std': [0.2353, 0.2157, 0.2289]},
    'foggy-cityscapes': {'mean': [0.4169, 0.4507, 0.4173], 'std': [0.1860, 0.1835, 0.1820]},
    'combined': {'mean': [0.5017, 0.5087, 0.4826], 'std': [0.2259, 0.2118, 0.2227]}
}

In [10]:
def dataloader_to_df(dataloader):
  X = []
  y = []
  for images, labels in dataloader:
    for image, label in zip(images, labels):
      X.append(image.numpy().reshape(-1))
      y.append(label.numpy().item())
  df = pd.DataFrame({'features': X, 'class': y})
  return df

In [11]:
set(DATASET_PATHS.keys()) - {'combined'}

{'fog-detection', 'fog-or-smog', 'foggy-cityscapes'}

In [12]:
dfs_color = {
    dataset: {
        'train': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'train', transforms["color"])),
        'val': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'val', transforms["color"])),
        'test': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'test', transforms["color"]))
    } for dataset in (set(DATASET_PATHS.keys()) - {'combined'})
}

# dfs_grayscale = {
#     dataset: {
#         'train': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'train', transforms["grayscale"])),
#         'val': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'val', transforms["grayscale"])),
#         'test': dataloader_to_df(get_dataloader(DATASET_PATHS[dataset] / 'test', transforms["grayscale"]))
#     } for dataset in (set(DATASET_PATHS.keys()) - {'combined'})
# }

In [13]:
dfs_color['combined'] = {}
for split in ['train', 'val', 'test']:
  dfs_combined = [dfs_color[dataset][split] for dataset in (set(DATASET_PATHS.keys()) - {'combined'})]
  dfs_combined = pd.concat(dfs_combined)
  dfs_color['combined'][split] = dfs_combined

In [14]:
_T = TypeVar('_T', bound=ClassifierMixin)

def evaluate_classic_model(model: _T, X: Any, y: Any):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')
    confusion = confusion_matrix(y, y_pred)
    return namedtuple('Evaluation', ['accuracy', 'f1', 'precision', 'recall', 'confusion'])(accuracy, f1, precision, recall, confusion)

def _get_formatted_metric(values: list[float]):
  avg = sum(values) / len(values)
  std = (sum([(v - avg) ** 2 for v in values]) / len(values)) ** 0.5
  return f"{avg:.4f} ± {std:.4f}"

def train_model(
    model_cls: _T,
    model_kwargs: dict[str, Any],
    train_df: pd.DataFrame,
) -> _T:
    model = model_cls(**model_kwargs)
    model.fit(np.stack(train_df['features'].values), train_df['class'])
    return model

def run_model(
    model_cls: _T,
    model_kwargs: dict[str, Any],
    train_df: pd.DataFrame,
    # val_df: pd.DataFrame,
    test_dfs: dict[str, pd.DataFrame],
    repeat: int = 5,
    transform: str = "color",
    normalize: bool = False
):
  final_res = {
      dataset: {
          'accuracy': [],
          'f1': [],
          'precision': [],
          'recall': [],
      } for dataset in test_dfs.keys()
  }
  training_times = []
  for i in range(repeat):

    _start = time()
    model = train_model(
        model_cls=model_cls,
        model_kwargs=model_kwargs,
        train_df=train_df,
        # val_df=val_df
    )
    _end = time()
    training_times.append(_end - _start)
    for dataset, test_df in test_dfs.items():
      results = evaluate_classic_model(
          model=model,
          X=np.stack(test_df['features'].values),
          y=test_df['class']
      )
      final_res[dataset]['accuracy'].append(results.accuracy)
      final_res[dataset]['f1'].append(results.f1)
      final_res[dataset]['precision'].append(results.precision)
      final_res[dataset]['recall'].append(results.recall)
  print(final_res)
  latex_table = [
    [
      dataset,
      *[_get_formatted_metric(m) for m in metrics.values()]
    ] for dataset, metrics in final_res.items()
  ]
  latex_table_str = ""
  for line in latex_table:
    latex_table_str += " & ".join([str(l) for l in line]) + " \\\\\n"

  return latex_table_str


# Ray Tune

In [15]:
import ray
from ray import tune

In [16]:
def subsample_df(df: pd.DataFrame, frac: float, target_column: str='class') -> pd.DataFrame:
  dfs = []
  for class_name in df[target_column].unique():
    class_df = df[df[target_column] == class_name]
    class_df = class_df.sample(frac=frac)
    dfs.append(class_df)
  return pd.concat(dfs)

In [17]:
def train_model_tune(
    config,
    model_cls: _T,
    train_df: pd.DataFrame,
    val_df: pd.DataFrame
):
  model = model_cls(**config)

  model.fit(np.stack(train_df['features'].values), train_df['class'])
  val_metrics = evaluate_classic_model(model, np.stack(val_df['features'].values), val_df['class'])

  tune.report(
      {
    "accuracy": val_metrics.accuracy,
    "f1": val_metrics.f1,
    "precision": val_metrics.precision,
    "recall": val_metrics.recall
      }
  )

# SVC

In [22]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=LinearSVC,
        train_df=subsample_df(dfs_color['combined']['train'], frac=0.05),
        val_df=dfs_color['combined']['val']
    ),
    param_space={
        "C": tune.grid_search([0.01, 0.1, 0.4, 0.8, 1, 1.2, 2.0, 10]),
    }
)

results = tuner.fit()

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_23-47-44   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 8                                      |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_23-47-44
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_22-18-50_111809_13588/artifacts/2025-06-07_23-47-44/train_model_tune_2025-06-07_23-47-44/driver_artifacts`

Trial status: 8 PENDING
Current time: 2025-06-07 23:47:44. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+-------------------------------------------------+


(train_model_tune pid=42558) /usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
(train_model_tune pid=42558)   warnings.warn(



Trial train_model_tune_cef85_00003 completed after 1 iterations at 2025-06-07 23:48:44. Total running time: 1min 0s
+-------------------------------------------------------+
| Trial train_model_tune_cef85_00003 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              56.0662 |
| time_total_s                                  56.0662 |
| training_iteration                                  1 |
| accuracy                                       0.6514 |
| f1                                            0.62058 |
| precision                                     0.69067 |
| recall                                        0.63912 |
+-------------------------------------------------------+

Trial status: 7 RUNNING | 1 TERMINATED
Current time: 2025-06-07 23:48:45. Total running time: 1min 0s
Logical resource usage: 7.0/8 CPUs, 0/0 GPUs
+--------------------------------------

2025-06-07 23:48:47,785	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_23-47-44' in 0.0090s.



Trial train_model_tune_cef85_00000 completed after 1 iterations at 2025-06-07 23:48:47. Total running time: 1min 3s
+-------------------------------------------------------+
| Trial train_model_tune_cef85_00000 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              59.1365 |
| time_total_s                                  59.1365 |
| training_iteration                                  1 |
| accuracy                                      0.66921 |
| f1                                            0.63704 |
| precision                                     0.72261 |
| recall                                        0.65635 |
+-------------------------------------------------------+

Trial status: 8 TERMINATED
Current time: 2025-06-07 23:48:47. Total running time: 1min 3s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
+--------------------------------------------------

In [23]:
best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=LinearSVC,
    model_kwargs=best_results.config,
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=1
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'fog-detection': {'accuracy': [0.8584905660377359], 'f1': [0.8363355635615028], 'precision': [0.90625], 'recall': [0.8170731707317074]}, 'fog-or-smog': {'accuracy': [0.8728448275862069], 'f1': [0.8711638829669579], 'precision': [0.8839456360161393], 'recall': [0.8699404761904762]}, 'foggy-cityscapes': {'accuracy': [0.6666666666666666], 'f1': [0.6565391084945332], 'precision': [0.6889531521110468], 'recall': [0.6666666666666666]}, 'combined': {'accuracy': [0.8177083333333334], 'f1': [0.812665955800874], 'precision': [0.8361327264718837], 'recall': [0.8113643781960613]}}


'fog-detection & 0.8585 ± 0.0000 & 0.8363 ± 0.0000 & 0.9062 ± 0.0000 & 0.8171 ± 0.0000 \\\\\nfog-or-smog & 0.8728 ± 0.0000 & 0.8712 ± 0.0000 & 0.8839 ± 0.0000 & 0.8699 ± 0.0000 \\\\\nfoggy-cityscapes & 0.6667 ± 0.0000 & 0.6565 ± 0.0000 & 0.6890 ± 0.0000 & 0.6667 ± 0.0000 \\\\\ncombined & 0.8177 ± 0.0000 & 0.8127 ± 0.0000 & 0.8361 ± 0.0000 & 0.8114 ± 0.0000 \\\\\n'

In [18]:
# best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=LinearSVC,
    # model_kwargs=best_results.config,
    model_kwargs={"C": 0.01},
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=5
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


{'foggy-cityscapes': {'accuracy': [0.6717171717171717, 0.6717171717171717, 0.6717171717171717, 0.6616161616161617, 0.6717171717171717], 'f1': [0.6623376623376623, 0.6623376623376623, 0.6623376623376623, 0.6531129879978035, 0.6623376623376623], 'precision': [0.6931818181818181, 0.6931818181818181, 0.6931818181818181, 0.679185520361991, 0.6931818181818181], 'recall': [0.6717171717171717, 0.6717171717171717, 0.6717171717171717, 0.6616161616161617, 0.6717171717171717]}, 'fog-detection': {'accuracy': [0.8584905660377359, 0.8584905660377359, 0.8584905660377359, 0.8584905660377359, 0.8584905660377359], 'f1': [0.8363355635615028, 0.8363355635615028, 0.8363355635615028, 0.8363355635615028, 0.8363355635615028], 'precision': [0.90625, 0.90625, 0.90625, 0.90625, 0.90625], 'recall': [0.8170731707317074, 0.8170731707317074, 0.8170731707317074, 0.8170731707317074, 0.8170731707317074]}, 'fog-or-smog': {'accuracy': [0.8728448275862069, 0.8728448275862069, 0.8728448275862069, 0.875, 0.875], 'f1': [0.871

'foggy-cityscapes & 0.6697 ± 0.0040 & 0.6605 ± 0.0037 & 0.6904 ± 0.0056 & 0.6697 ± 0.0040 \\\\\nfog-detection & 0.8585 ± 0.0000 & 0.8363 ± 0.0000 & 0.9062 ± 0.0000 & 0.8171 ± 0.0000 \\\\\nfog-or-smog & 0.8737 ± 0.0011 & 0.8722 ± 0.0011 & 0.8837 ± 0.0008 & 0.8710 ± 0.0011 \\\\\ncombined & 0.8190 ± 0.0008 & 0.8143 ± 0.0008 & 0.8361 ± 0.0013 & 0.8129 ± 0.0008 \\\\\n'

# RFC

In [18]:
tuner = tune.Tuner(
    tune.with_parameters(
        train_model_tune,
        model_cls=RandomForestClassifier,
        # train_df=dfs_color['combined']['train'],
        train_df=subsample_df(dfs_color['combined']['train'], frac=0.2),
        val_df=dfs_color['combined']['val']
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 100, 200]),
        "max_depth": tune.grid_search([1, 5, 10, 20, 50, None]),
    }
)

results = tuner.fit()

2025-06-07 22:18:51,551	INFO worker.py:1888 -- Started a local Ray instance.
2025-06-07 22:18:54,641	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_22-18-50   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 24                                     |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_22-18-50
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_22-18-50_111809_13588/artifacts/2025-06-07_22-18-54/train_model_tune_2025-06-07_22-18-50/driver_artifacts`

Trial status: 24 PENDING
Current time: 2025-06-07 22:18:55. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+--------------------------------------------------

2025-06-07 22:19:38,556	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_22-18-50' in 0.0095s.



Trial train_model_tune_6607c_00023 completed after 1 iterations at 2025-06-07 22:19:38. Total running time: 43s
+-------------------------------------------------------+
| Trial train_model_tune_6607c_00023 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                               16.798 |
| time_total_s                                   16.798 |
| training_iteration                                  1 |
| accuracy                                      0.85242 |
| f1                                            0.85226 |
| precision                                      0.8521 |
| recall                                        0.85281 |
+-------------------------------------------------------+

Trial status: 24 TERMINATED
Current time: 2025-06-07 22:19:38. Total running time: 43s
Logical resource usage: 1.0/8 CPUs, 0/0 GPUs
+---------------------------------------------------------

In [19]:
best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=RandomForestClassifier,
    model_kwargs=best_results.config,
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=5
)

{'fog-detection': {'accuracy': [0.9811320754716981, 0.9905660377358491, 0.9905660377358491, 0.9905660377358491, 1.0], 'f1': [0.9799242424242425, 0.9900103666006974, 0.9900103666006974, 0.9900103666006974, 1.0], 'precision': [0.9850746268656716, 0.9924242424242424, 0.9924242424242424, 0.9924242424242424, 1.0], 'recall': [0.975609756097561, 0.9878048780487805, 0.9878048780487805, 0.9878048780487805, 1.0]}, 'fog-or-smog': {'accuracy': [0.9504310344827587, 0.9375, 0.9525862068965517, 0.9418103448275862, 0.9418103448275862], 'f1': [0.9504289622689712, 0.9374927417069647, 0.9525826830174656, 0.9418079122287923, 0.9418035871064844], 'precision': [0.9510121005966654, 0.9377961534888042, 0.9530181385667559, 0.9423874049703527, 0.9421072191768094], 'recall': [0.9513392857142857, 0.9382440476190477, 0.953422619047619, 0.9427083333333333, 0.9425595238095238]}, 'foggy-cityscapes': {'accuracy': [0.8131313131313131, 0.8181818181818182, 0.8333333333333334, 0.8131313131313131, 0.8181818181818182], 'f1'

'fog-detection & 0.9906 ± 0.0060 & 0.9900 ± 0.0063 & 0.9925 ± 0.0047 & 0.9878 ± 0.0077 \\\\\nfog-or-smog & 0.9448 ± 0.0057 & 0.9448 ± 0.0057 & 0.9453 ± 0.0058 & 0.9457 ± 0.0058 \\\\\nfoggy-cityscapes & 0.8192 ± 0.0074 & 0.8180 ± 0.0074 & 0.8280 ± 0.0081 & 0.8192 ± 0.0074 \\\\\ncombined & 0.9187 ± 0.0046 & 0.9187 ± 0.0046 & 0.9194 ± 0.0046 & 0.9204 ± 0.0046 \\\\\n'

# XGBoost

In [20]:
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(
            train_model_tune,
            model_cls=XGBClassifier,
            train_df=subsample_df(dfs_color['combined']['train'], frac=0.2),
            val_df=dfs_color['combined']['val']
        ),
        resources={"cpu": 8}
    ),
    param_space={
        "n_estimators": tune.grid_search([10, 50, 100, 200]),
        "max_depth": tune.grid_search([1, 5, 10, 20, 50, None]),

    }
)

results = tuner.fit()

+-------------------------------------------------------------------------+
| Configuration for experiment     train_model_tune_2025-06-07_22-33-37   |
+-------------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                  |
| Scheduler                        FIFOScheduler                          |
| Number of trials                 24                                     |
+-------------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_model_tune_2025-06-07_22-33-37
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-07_22-18-50_111809_13588/artifacts/2025-06-07_22-33-37/train_model_tune_2025-06-07_22-33-37/driver_artifacts`

Trial status: 24 PENDING
Current time: 2025-06-07 22:33:38. Total running time: 0s
Logical resource usage: 0/8 CPUs, 0/0 GPUs
+--------------------------------------------------

2025-06-07 23:20:57,551	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_model_tune_2025-06-07_22-33-37' in 0.0082s.



Trial train_model_tune_7476c_00023 completed after 1 iterations at 2025-06-07 23:20:57. Total running time: 47min 19s
+-------------------------------------------------------+
| Trial train_model_tune_7476c_00023 result             |
+-------------------------------------------------------+
| checkpoint_dir_name                                   |
| time_this_iter_s                              195.046 |
| time_total_s                                  195.046 |
| training_iteration                                  1 |
| accuracy                                      0.86514 |
| f1                                            0.86464 |
| precision                                     0.86535 |
| recall                                        0.86421 |
+-------------------------------------------------------+

Trial status: 24 TERMINATED
Current time: 2025-06-07 23:20:57. Total running time: 47min 19s
Logical resource usage: 8.0/8 CPUs, 0/0 GPUs
+---------------------------------------------

In [21]:
best_results = results.get_best_result(metric='f1', mode='max')

run_model(
    model_cls=XGBClassifier,
    model_kwargs=best_results.config,
    train_df=dfs_color['combined']['train'],
    test_dfs={dataset: dfs_color[dataset]['test'] for dataset in dfs_color.keys()},
    repeat=5
)

{'fog-detection': {'accuracy': [0.9716981132075472, 0.9716981132075472, 0.9716981132075472, 0.9716981132075472, 0.9716981132075472], 'f1': [0.9697344627391262, 0.9697344627391262, 0.9697344627391262, 0.9697344627391262, 0.9697344627391262], 'precision': [0.9779411764705883, 0.9779411764705883, 0.9779411764705883, 0.9779411764705883, 0.9779411764705883], 'recall': [0.9634146341463414, 0.9634146341463414, 0.9634146341463414, 0.9634146341463414, 0.9634146341463414]}, 'fog-or-smog': {'accuracy': [0.9418103448275862, 0.9418103448275862, 0.9418103448275862, 0.9418103448275862, 0.9418103448275862], 'f1': [0.9417494687800773, 0.9417494687800773, 0.9417494687800773, 0.9417494687800773, 0.9417494687800773], 'precision': [0.9416922361692237, 0.9416922361692237, 0.9416922361692237, 0.9416922361692237, 0.9416922361692237], 'recall': [0.9418154761904762, 0.9418154761904762, 0.9418154761904762, 0.9418154761904762, 0.9418154761904762]}, 'foggy-cityscapes': {'accuracy': [0.8484848484848485, 0.848484848

'fog-detection & 0.9717 ± 0.0000 & 0.9697 ± 0.0000 & 0.9779 ± 0.0000 & 0.9634 ± 0.0000 \\\\\nfog-or-smog & 0.9418 ± 0.0000 & 0.9417 ± 0.0000 & 0.9417 ± 0.0000 & 0.9418 ± 0.0000 \\\\\nfoggy-cityscapes & 0.8485 ± 0.0000 & 0.8483 ± 0.0000 & 0.8498 ± 0.0000 & 0.8485 ± 0.0000 \\\\\ncombined & 0.9219 ± 0.0000 & 0.9217 ± 0.0000 & 0.9215 ± 0.0000 & 0.9219 ± 0.0000 \\\\\n'

## Not Deep models

In [ ]:
run_model(
    model_cls=LinearSVC,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=1
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9981
		F1: 0.9981
		Precision: 0.9982
		Recall: 0.9981
	Validation metrics:
		Accuracy: 0.7888
		F1: 0.7797
		Precision: 0.8215
		Recall: 0.7808


'combined & 0.8060 ± 0.0000 & 0.7988 ± 0.0000 & 0.8328 ± 0.0000 & 0.7983 ± 0.0000 \\\\\n'

In [ ]:
run_model(
    model_cls=LinearSVC,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=5
)

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9989
		F1: 0.9989
		Precision: 0.9989
		Recall: 0.9989
	Validation metrics:
		Accuracy: 0.7939
		F1: 0.7859
		Precision: 0.8227
		Recall: 0.7864


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9993
		F1: 0.9993
		Precision: 0.9993
		Recall: 0.9992
	Validation metrics:
		Accuracy: 0.7888
		F1: 0.7801
		Precision: 0.8192
		Recall: 0.7810


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9981
		F1: 0.9981
		Precision: 0.9981
		Recall: 0.9982
	Validation metrics:
		Accuracy: 0.7964
		F1: 0.7907
		Precision: 0.8147
		Recall: 0.7903


/usr/local/lib/python3.11/dist-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Model - LinearSVC
	Train metrics:
		Accuracy: 0.9981
		F1: 0.9981
		Precision: 0.9982
		Recall: 0.9981
	Validation metrics:
		Accuracy: 0.7913
		F1: 0.7843
		Precision: 0.8146
		Recall: 0.7844


'combined & 0.8128 ± 0.0036 & 0.8071 ± 0.0042 & 0.8333 ± 0.0018 & 0.8060 ± 0.0040 \\\\\n'

In [ ]:
run_model(
    model_cls=RandomForestClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=1
)

Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9288
		F1: 0.9287
		Precision: 0.9286
		Recall: 0.9296


'combined & 0.9128 ± 0.0000 & 0.9127 ± 0.0000 & 0.9136 ± 0.0000 & 0.9145 ± 0.0000 \\\\\n'

In [ ]:
run_model(
    model_cls=RandomForestClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=5
)

Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9237
		F1: 0.9236
		Precision: 0.9235
		Recall: 0.9245
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9186
		F1: 0.9185
		Precision: 0.9184
		Recall: 0.9194
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9186
		F1: 0.9185
		Precision: 0.9182
		Recall: 0.9189
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9237
		F1: 0.9236
		Precision: 0.9234
		Recall: 0.9242
Model - RandomForestClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9211
		F1: 0.9210
		Preci

'combined & 0.9193 ± 0.0044 & 0.9193 ± 0.0044 & 0.9201 ± 0.0042 & 0.9210 ± 0.0042 \\\\\n'

In [ ]:
run_model(
    model_cls=XGBClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=1
)

Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9389
		F1: 0.9388
		Precision: 0.9386
		Recall: 0.9390


'combined & 0.9284 ± 0.0000 & 0.9282 ± 0.0000 & 0.9281 ± 0.0000 & 0.9283 ± 0.0000 \\\\\n'

In [ ]:
run_model(
    model_cls=XGBClassifier,
    model_kwargs={},
    train_df=train_df,
    val_df=val_df,
    test_df=test_df,
    repeat=5
)

Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9389
		F1: 0.9388
		Precision: 0.9386
		Recall: 0.9390
Model - XGBClassifier
	Train metrics:
		Accuracy: 1.0000
		F1: 1.0000
		Precision: 1.0000
		Recall: 1.0000
	Validation metrics:
		Accuracy: 0.9389
		F1: 0.9388
		Precision: 0.9386
		Recall: 0.9390
